In [1]:
import wrangle as w
import new_lib as nl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from math import sqrt
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
zil = w.wrangle_zillow()
zil
# Wrangle the data into workable format 

,id,bedrooms,bathrooms,square_footage,lot_size,tax_value,yearbuilt,fips
index,,,,,,,,
0,1727539,4.0,3.5,3100.0,4506.0,1023282.0,1998,6059.0
1,1387261,2.0,1.0,1465.0,12647.0,464000.0,1967,6111.0
2,11677,3.0,2.0,1243.0,8432.0,564778.0,1962,6059.0
3,2288172,4.0,3.0,2376.0,13038.0,145143.0,1970,6037.0
4,781532,4.0,3.0,2962.0,63000.0,773303.0,1950,6037.0
...,...,...,...,...,...,...,...,...
51955,673515,2.0,2.0,1286.0,47405.0,354621.0,1940,6037.0
51956,2968375,4.0,2.0,1612.0,12105.0,67205.0,1964,6111.0
51957,1843709,3.0,1.0,1032.0,5074.0,49546.0,1954,6037.0


In [3]:
zil, fences = nl.remove_outliers(zil)
# Remove any outliers from data

fips column ignored


In [4]:
zil['room_count'] = zil['bedrooms'] + zil['bathrooms']
zil = zil.drop(columns = ['bedrooms', 'bathrooms'])
zil.head()
# Combining the rooms into one feature for modeling

,id,square_footage,lot_size,tax_value,yearbuilt,fips,room_count
index,,,,,,,
0,1727539,3100.0,4506.0,1023282.0,1998,6059.0,7.5
1,1387261,1465.0,12647.0,464000.0,1967,6111.0,3.0
2,11677,1243.0,8432.0,564778.0,1962,6059.0,5.0
3,2288172,2376.0,13038.0,145143.0,1970,6037.0,7.0
5,870991,738.0,4214.0,218552.0,1922,6037.0,3.0


In [5]:
train, validate, test, X_train, y_train, X_val, y_val, X_test, y_test = nl.train_vailidate_test_split(zil, 'tax_value')
# Creating a split for the data

In [6]:
train_scaled, val_scaled, test_scaled = nl.scale_splits(X_train, X_val, X_test, MinMaxScaler(), ['room_count', 'square_footage', 'lot_size', 'yearbuilt'])
train_scaled.head()
# Scaling data and confirming result

,id,square_footage,lot_size,yearbuilt,fips,room_count
index,,,,,,
18573,2138356,0.320749,0.429523,0.550459,6037.0,0.307692
32318,2695053,0.274198,0.406632,0.513761,6059.0,0.307692
17200,145241,0.411581,0.574161,0.486239,6059.0,0.307692
570,643458,0.241272,0.370100,0.311927,6037.0,0.000000
32875,1144215,0.507806,0.516384,0.495413,6059.0,0.538462


In [7]:
train_scaled = pd.get_dummies(train_scaled, columns= ['fips'])
train_scaled.head()
# Getting dummies for fips which is the only catagorical variable

,id,square_footage,lot_size,yearbuilt,room_count,fips_6037.0,fips_6059.0,fips_6111.0
index,,,,,,,,
18573,2138356,0.320749,0.429523,0.550459,0.307692,1,0,0
32318,2695053,0.274198,0.406632,0.513761,0.307692,0,1,0
17200,145241,0.411581,0.574161,0.486239,0.307692,0,1,0
570,643458,0.241272,0.370100,0.311927,0.000000,1,0,0
32875,1144215,0.507806,0.516384,0.495413,0.538462,0,1,0


In [8]:
lm = LinearRegression()
lm.fit(train_scaled, y_train)
# Setting up linear regression

LinearRegression()

In [9]:
lm_preds = lm.predict(train_scaled)
lm_preds

array([313121.49545663, 336133.26382967, 431471.2039668 , ...,
       311228.91270278, 530850.99153134, 265015.23675672])

In [10]:
preds = pd.DataFrame({'actual':y_train,
                      'baseline':y_train.mean(),
                      'lm_preds':lm_preds})
preds
# Getting predicitons

,actual,baseline,lm_preds
index,,,
18573,269997.0,380044.138663,313121.495457
32318,68248.0,380044.138663,336133.263830
17200,271093.0,380044.138663,431471.203967
570,39307.0,380044.138663,306083.675254
32875,492428.0,380044.138663,487939.447215
...,...,...,...
11025,701000.0,380044.138663,470057.116338
31513,710674.0,380044.138663,431206.419911
18178,923877.0,380044.138663,311228.912703


In [11]:
sqrt(mean_squared_error(preds['actual'], preds['lm_preds']))
# Grabbing an rfe to assess model

224432.98779335924

In [12]:
lasso = LassoLars(alpha = .1)
lasso.fit(train_scaled, y_train)

lasso_preds = lasso.predict(train_scaled)
# Lasso Lars setup

In [13]:
preds = pd.DataFrame({'actual':y_train,
                      'baseline':y_train.mean(),
                      'lasso_preds':lasso_preds})
preds

,actual,baseline,lasso_preds
index,,,
18573,269997.0,380044.138663,313168.915008
32318,68248.0,380044.138663,336161.992637
17200,271093.0,380044.138663,431408.018822
570,39307.0,380044.138663,306003.940790
32875,492428.0,380044.138663,487910.010228
...,...,...,...
11025,701000.0,380044.138663,470052.661602
31513,710674.0,380044.138663,431218.010186
18178,923877.0,380044.138663,311183.810999


In [14]:
sqrt(mean_squared_error(preds['actual'], preds['lasso_preds']))
# Rmse to assess

224432.9959271145

In [15]:
pol = PolynomialFeatures(degree=2)

pol.fit(train_scaled, y_train)
# Polynomial model setup

PolynomialFeatures()

In [16]:
poly = pol.transform(train_scaled)
lin = LinearRegression()

lin.fit(poly, y_train)
poly_preds = lin.predict(poly)

In [17]:
preds = pd.DataFrame({'actual':y_train,
                      'baseline':y_train.mean(),
                      'poly_preds':poly_preds})
preds

,actual,baseline,poly_preds
index,,,
18573,269997.0,380044.138663,285075.004110
32318,68248.0,380044.138663,321379.322095
17200,271093.0,380044.138663,401483.050985
570,39307.0,380044.138663,293903.721974
32875,492428.0,380044.138663,407385.115027
...,...,...,...
11025,701000.0,380044.138663,481566.526885
31513,710674.0,380044.138663,399351.090770
18178,923877.0,380044.138663,304239.485435


In [18]:
sqrt(mean_squared_error(preds['actual'], preds['poly_preds']))
# Assessing polynomial model

219290.68576087515

In [19]:
pol_val = PolynomialFeatures(degree=2)

pol_val.fit(val_scaled, y_train)
# running validate setup

PolynomialFeatures()

In [20]:
poly_val = pol_val.transform(val_scaled)
lin_val = LinearRegression()

lin_val.fit(poly_val, y_val)
val_preds = lin_val.predict(poly_val)
# validate predictions

In [21]:
val_preds = pd.DataFrame({'actual':y_val,
                      'baseline':y_val.mean(),
                      'val_preds':val_preds})
val_preds

,actual,baseline,val_preds
index,,,
34680,261096.0,378892.414403,216319.332141
29476,122000.0,378892.414403,281045.238487
9203,730000.0,378892.414403,471725.380689
37354,553031.0,378892.414403,277516.296832
36631,267292.0,378892.414403,312465.657063
...,...,...,...
5050,234049.0,378892.414403,273694.476913
38955,717557.0,378892.414403,642205.950768
14709,216666.0,378892.414403,376294.816243


In [22]:
sqrt(mean_squared_error(val_preds['actual'], val_preds['val_preds']))
# results from the model

218171.81376901644

In [23]:
lasso_v = LassoLars(alpha = .1)
lasso_v.fit(val_scaled, y_val)

preds_v = lasso_v.predict(val_scaled)
# lasso lars setup for the validate

In [24]:
preds = pd.DataFrame({'actual':y_val,
                      'baseline':y_val.mean(),
                      'lasso_val':preds_v})
preds

,actual,baseline,lasso_val
index,,,
34680,261096.0,378892.414403,218511.677845
29476,122000.0,378892.414403,288691.102729
9203,730000.0,378892.414403,483865.810898
37354,553031.0,378892.414403,286832.494502
36631,267292.0,378892.414403,344633.285041
...,...,...,...
5050,234049.0,378892.414403,198909.719737
38955,717557.0,378892.414403,613631.208222
14709,216666.0,378892.414403,388999.972926


In [25]:
sqrt(mean_squared_error(preds['actual'], preds['lasso_val']))
# lasso lars results on the validate set

220632.5875462855

In [26]:
lm_v = LinearRegression()
lm_v.fit(val_scaled, y_val)

LinearRegression()

In [27]:
val_preds = lm_v.predict(val_scaled)
val_preds

array([218476.92081967, 288611.24932009, 483834.70723044, ...,
       389051.70566716, 802222.47053409, 387166.23206443])

In [28]:
lmv_preds = pd.DataFrame({'actual':y_val,
                      'baseline':y_val.mean(),
                      'lm_v':val_preds})
lmv_preds

,actual,baseline,lm_v
index,,,
34680,261096.0,378892.414403,218476.920820
29476,122000.0,378892.414403,288611.249320
9203,730000.0,378892.414403,483834.707230
37354,553031.0,378892.414403,286834.369368
36631,267292.0,378892.414403,344591.939942
...,...,...,...
5050,234049.0,378892.414403,198885.136566
38955,717557.0,378892.414403,613662.476350
14709,216666.0,378892.414403,389051.705667


In [29]:
sqrt(mean_squared_error(lmv_preds['actual'], lmv_preds['lm_v']))
# results for linear regression 

220632.58381914545

In [30]:
sqrt(mean_squared_error(lmv_preds['actual'], lmv_preds['baseline']))
# grabbing baseline results to compare the models against

253057.52641864202

In [31]:
pol_test = PolynomialFeatures(degree=2)

pol_test.fit(test_scaled, y_test)


PolynomialFeatures()

In [32]:
poly_test = pol_test.transform(test_scaled)
lin_test = LinearRegression()

lin_test.fit(poly_test, y_test)
test_preds = lin_test.predict(poly_test)
# Deciding polynomial was best and setting up for the test set

In [33]:
test_preds = pd.DataFrame({'actual':y_test,
                      'baseline':y_test.mean(),
                      'test_preds':test_preds})
test_preds

,actual,baseline,test_preds
index,,,
29516,341016.0,377473.606942,22785.229797
5322,395083.0,377473.606942,521237.397773
38993,722000.0,377473.606942,392777.439481
11465,171326.0,377473.606942,265731.387046
21105,296484.0,377473.606942,330769.402480
...,...,...,...
50569,713063.0,377473.606942,281774.732614
12821,292300.0,377473.606942,310901.679232
31130,521000.0,377473.606942,455270.937881


In [34]:
sqrt(mean_squared_error(test_preds['actual'], test_preds['test_preds']))
# results of the test model

229865.01324872902